# About: OperationHubのインストール

# 各種設定

## インストール対象VM

In [3]:
target_group = '-i ./hosts compute'

!ansible -m ping {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | SUCCESS => {
    "changed": false,
    "ping": "pong"
}


In [4]:
ophub_dir = '/opt/ophub'

!ansible -b -m file -a 'path={ophub_dir} state=directory' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED => {
    "changed": true,
    "gid": 0,
    "group": "root",
    "mode": "0755",
    "owner": "root",
    "path": "/opt/ophub",
    "secontext": "unconfined_u:object_r:usr_t:s0",
    "size": 6,
    "state": "directory",
    "uid": 0
}


対象ホストはCentOS 7がデプロイされているものとします。

In [5]:
!ansible -a 'cat /etc/centos-release' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
CentOS Linux release 7.7.1908 (Core)


## ドメイン名設定

`server_name`変数に、OperationHubを動作させるホストのドメイン名を設定する。

ここではデモ用と限定して、AWSで確保されたインスタンスである前提で設定する。

In [6]:
import re

ping_result = !ansible -m ping {target_group}
ec2_domain_pattern = re.compile(r'(ec2-.+\..+\.compute\.amazonaws\.com)')
server_name = None
for line in ping_result:
    m = ec2_domain_pattern.search(line)
    if m:
        server_name = m.group(1)

assert server_name is not None, 'Unexpected result: {}'.format(''.join(ping_result))
server_name

'ec2-52-42-214-180.us-west-2.compute.amazonaws.com'

## サーバ証明書設定

`cert_file`, `key_file` 変数に、サーバ証明書と秘密鍵のパス(Notebookサーバ上のパス)を指定する。

ここではデモ用と限定して、自己署名の証明書を設定する。

In [7]:
import os

key_file = '.cert-20200702/demo.key'

!mkdir -p {os.path.split(key_file)[0]}
!openssl genrsa 2048 > {key_file}

Generating RSA private key, 2048 bit long modulus (2 primes)
.........................................................................................................................................................+++++
.........+++++
e is 65537 (0x010001)


In [9]:
cert_file = '.cert-20200702/demo.cer'

!openssl req -new -subj "/C=JP/CN={server_name}" -x509 -days 31 -key {key_file} -sha512 -out {cert_file}

内容を確認する。

In [10]:
!openssl x509 -in {cert_file} -noout -text

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            0e:30:3e:66:68:9e:ba:f2:7f:ac:3e:0f:e0:8a:2b:35:48:de:e9:f3
        Signature Algorithm: sha512WithRSAEncryption
        Issuer: C = JP, CN = ec2-52-42-214-180.us-west-2.compute.amazonaws.com
        Validity
            Not Before: Jul  2 02:17:29 2020 GMT
            Not After : Aug  2 02:17:29 2020 GMT
        Subject: C = JP, CN = ec2-52-42-214-180.us-west-2.compute.amazonaws.com
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                RSA Public-Key: (2048 bit)
                Modulus:
                    00:b2:51:66:20:b3:18:37:46:fb:97:0a:db:af:88:
                    e0:f3:8b:b6:25:83:5b:0f:b6:3d:13:d9:a5:83:55:
                    f6:cf:48:c2:5d:4b:e0:4d:15:54:3c:4d:eb:59:62:
                    7d:f0:e3:91:e0:72:d1:05:b1:71:7d:6b:2d:7b:ce:
                    12:84:05:d0:fd:de:c4:a9:90:38:18:cf:24:d6:ed:
                    96:5d:05:f5:c4:55:d4:4b:ef:87:c

## 管理者ユーザ設定

最初に作る管理者ユーザのユーザ名及びパスワードを指定する。

PasswordはNotebookに残らないよう、getpassを使用して入力する。

In [24]:
import getpass

admin_username = 'john'
admin_password = getpass.getpass()

········


# インストールの実施

https://github.com/NII-cloud-operation/OperationHub に従いながら...

## Step 1: Download OperationHub files

まず、対象のホストにgitをインストールする。

In [11]:
!ansible -b -m yum -a 'name=git' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED => {
    "ansible_facts": {
        "pkg_mgr": "yum"
    },
    "changed": true,
    "changes": {
        "installed": [
            "git"
        ]
    },
    "msg": "warning: /var/cache/yum/x86_64/7/base/packages/perl-Carp-1.26-244.el7.noarch.rpm: Header V3 RSA/SHA256 Signature, key ID f4a80eb5: NOKEY\nImporting GPG key 0xF4A80EB5:\n Userid     : \"CentOS-7 Key (CentOS 7 Official Signing Key) <security@centos.org>\"\n Fingerprint: 6341 ab27 53d7 8a78 a7c2 7bb1 24c6 a8a7 f4a8 0eb5\n Package    : centos-release-7-7.1908.0.el7.centos.x86_64 (installed)\n From       : /etc/pki/rpm-gpg/RPM-GPG-KEY-CentOS-7\n",
    "rc": 0,
    "results": [
        "Loaded plugins: fastestmirror\nLoading mirror speeds from cached hostfile\n * base: d36uatko69830t.cloudfront.net\n * extras: d36uatko69830t.cloudfront.net\n * updates: d36uatko69830t.cloudfront.net\nResolving Dependencies\n--> Running transaction check\n---> Package git.x86_64 0:1.8.3

リポジトリのcloneはAnsible gitモジュールを使う。

In [12]:
!ansible -b -m git -a 'repo=https://github.com/NII-cloud-operation/OperationHub.git dest={ophub_dir}' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED => {
    "after": "7efb3c44bc8bd8c34344be39d1bd3ff3d4b00c2c",
    "before": null,
    "changed": true
}


cloneされたバージョンを押さえておこう... あとでうまく動作しなくなった！と言う時に良いヒントとなります。

In [13]:
!ansible -a 'ls -la {ophub_dir}' {target_group}
!ansible -a 'chdir={ophub_dir} git log -1' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
total 48
drwxr-xr-x. 6 root root   244 Jul  2 02:18 .
drwxr-xr-x. 3 root root    19 Jul  2 02:16 ..
-rw-r--r--. 1 root root  3127 Jul  2 02:18 docker-compose.yml
-rw-r--r--. 1 root root   566 Jul  2 02:18 .env.sample
drwxr-xr-x. 8 root root   180 Jul  2 02:18 .git
-rw-r--r--. 1 root root    14 Jul  2 02:18 .gitignore
drwxr-xr-x. 2 root root   126 Jul  2 02:18 host-service
drwxr-xr-x. 4 root root    37 Jul  2 02:18 images
-rwxr-xr-x. 1 root root   598 Jul  2 02:18 install-docker.sh
-rwxr-xr-x. 1 root root   972 Jul  2 02:18 install-host-services.sh
-rw-r--r--. 1 root root 10321 Jul  2 02:18 jupyterhub-logo.png
-rw-r--r--. 1 root root  1534 Jul  2 02:18 LICENSE
drwxr-xr-x. 2 root root    33 Jul  2 02:18 nginx
-rw-r--r--. 1 root root  8304 Jul  2 02:18 README.md
ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
commit 7efb3c44bc8bd8c34344be39d1bd3ff3d4b00c2c
Merge: f04d0b1 6f2bf53
Author: Shingo Naka

##  Step 2: Install Docker Engine and docker-compose

Docker EngineとDocker Composeを対象ホストにインストールする。

In [14]:
!ansible -b -a 'chdir={ophub_dir} ./install-docker.sh' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: d36uatko69830t.cloudfront.net
 * extras: d36uatko69830t.cloudfront.net
 * updates: d36uatko69830t.cloudfront.net
Resolving Dependencies
--> Running transaction check
---> Package device-mapper-persistent-data.x86_64 0:0.8.5-2.el7 will be installed
--> Processing Dependency: libaio.so.1(LIBAIO_0.4)(64bit) for package: device-mapper-persistent-data-0.8.5-2.el7.x86_64
--> Processing Dependency: libaio.so.1(LIBAIO_0.1)(64bit) for package: device-mapper-persistent-data-0.8.5-2.el7.x86_64
--> Processing Dependency: libaio.so.1()(64bit) for package: device-mapper-persistent-data-0.8.5-2.el7.x86_64
---> Package lvm2.x86_64 7:2.02.186-7.el7_8.2 will be installed
--> Processing Dependency: lvm2-libs = 7:2.02.186-7.el7_8.2 for package: 7:lvm2-2.02.186-7.el7_8.2.x86_64
--> Processing Dependency: liblvm2app.so.2.2(Base)(64bit) for package: 7:lvm2-2.

## Step 3: Install the host services for OperationHub

OperationHubに必要なサービスを対象ホストにインストールする。

In [15]:
!ansible -b -a 'chdir={ophub_dir} ./install-host-services.sh' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: d36uatko69830t.cloudfront.net
 * extras: d36uatko69830t.cloudfront.net
 * updates: d36uatko69830t.cloudfront.net
Resolving Dependencies
--> Running transaction check
---> Package epel-release.noarch 0:7-11 will be installed
--> Finished Dependency Resolution

Dependencies Resolved

 Package                Arch             Version         Repository        Size
Installing:
 epel-release           noarch           7-11            extras            15 k

Transaction Summary
Install  1 Package

Total download size: 15 k
Installed size: 24 k
Running transaction check
Running transaction test
Transaction test succeeded
Running transaction
  Installing : epel-release-7-11.noarch                                     1/1 
  Verifying  : epel-release-7-11.noarch                                     1/1 

Installed:
  epel-release.noarch 0:7-11     

## Step 4: Setting domain name

DNS名を設定します。

In [17]:
import tempfile

work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmpov_y4a_6'

In [16]:
!ansible -b -a 'chdir={ophub_dir} cat .env.sample' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
# Server Name for nginx TLS communication (reverse proxy)
SERVER_NAME=example.com

# JupyterHub single-user docker image
SINGLE_USER_IMAGE=niicloudoperation/notebook

# Configuration for culling idle servers
## Enable culling
CULL_SERVER=yes

## The idle timeout (in seconds) for culling server
CULL_SERVER_IDLE_TIMEOUT=300

## The maximum age (in seconds) of servers that should be culled even if they are active
CULL_SERVER_MAX_AGE=86400

## The interval (in seconds) for checking for idle servers to cull
CULL_SERVER_EVERY=60

# Enable Debug Logging
# DEBUG=yes


In [18]:
with open(os.path.join(work_dir, '.env'), 'w') as f:
    f.write('''# Server Name for nginx TLS communication (reverse proxy)
SERVER_NAME={server_name}

# JupyterHub single-user docker image
SINGLE_USER_IMAGE=niicloudoperation/notebook

# Configuration for culling idle servers
## Enable culling
CULL_SERVER=yes

## The idle timeout (in seconds) for culling server
CULL_SERVER_IDLE_TIMEOUT=300

## The maximum age (in seconds) of servers that should be culled even if they are active
CULL_SERVER_MAX_AGE=86400

## The interval (in seconds) for checking for idle servers to cull
CULL_SERVER_EVERY=60

# Enable Debug Logging
# DEBUG=yes
'''.format(**locals()))

!cat {work_dir}/.env

# Server Name for nginx TLS communication (reverse proxy)
SERVER_NAME=ec2-52-42-214-180.us-west-2.compute.amazonaws.com

# JupyterHub single-user docker image
SINGLE_USER_IMAGE=niicloudoperation/notebook

# Configuration for culling idle servers
## Enable culling
CULL_SERVER=yes

## The idle timeout (in seconds) for culling server
CULL_SERVER_IDLE_TIMEOUT=300

## The maximum age (in seconds) of servers that should be culled even if they are active
CULL_SERVER_MAX_AGE=86400

## The interval (in seconds) for checking for idle servers to cull
CULL_SERVER_EVERY=60

# Enable Debug Logging
# DEBUG=yes


In [20]:
!ansible -b -m copy -a 'src={work_dir}/.env dest={ophub_dir}/.env' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED => {
    "changed": true,
    "checksum": "a654bc7cae186140ddd81d02f9ee4f5ae8e7ae2d",
    "dest": "/opt/ophub/.env",
    "gid": 0,
    "group": "root",
    "md5sum": "99091c6b22298e01e95cc0746b9b5dae",
    "mode": "0644",
    "owner": "root",
    "secontext": "system_u:object_r:usr_t:s0",
    "size": 603,
    "src": "/home/centos/.ansible/tmp/ansible-tmp-1593656618.9467075-528-132812348185173/source",
    "state": "file",
    "uid": 0
}


## Step 5: Install TLS certificate and key

In [22]:
!ansible -b -m file -a 'path={ophub_dir}/cert state=directory' {target_group}
!ansible -b -m copy -a 'src={cert_file} dest={ophub_dir}/cert/server.cer' {target_group}
!ansible -b -m copy -a 'src={key_file} dest={ophub_dir}/cert/server.key' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED => {
    "changed": true,
    "gid": 0,
    "group": "root",
    "mode": "0755",
    "owner": "root",
    "path": "/opt/ophub/cert",
    "secontext": "unconfined_u:object_r:usr_t:s0",
    "size": 6,
    "state": "directory",
    "uid": 0
}
ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED => {
    "changed": true,
    "checksum": "51b947d932dddb69d9c2215157dce0faad7adda9",
    "dest": "/opt/ophub/cert/server.cer",
    "gid": 0,
    "group": "root",
    "md5sum": "0ee9443649f032253fb2a08b6006b5f8",
    "mode": "0644",
    "owner": "root",
    "secontext": "system_u:object_r:usr_t:s0",
    "size": 1257,
    "src": "/home/centos/.ansible/tmp/ansible-tmp-1593656735.809981-593-159453422361420/source",
    "state": "file",
    "uid": 0
}
ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED => {
    "changed": true,
    "checksum": "d284ecdad13933b7b214d1d43f771b9162df4d86",
    "dest": "/opt/ophub/cert/server.key",


## Step 6: Setting JupyterHub administrator

最初の管理者ユーザを設定する。ユーザ名とパスワードは 各種設定 - 管理者ユーザ設定 で実施されているものとする。

In [28]:
from passlib.hash import sha512_crypt

# 参考: https://docs.ansible.com/ansible/faq.html#how-do-i-generate-encrypted-passwords-for-the-user-module
!ansible -b -m user -a 'name={admin_username} password={sha512_crypt.using(rounds=5000).hash(admin_password)}' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED => {
    "changed": true,
    "comment": "",
    "create_home": true,
    "group": 1001,
    "home": "/home/yazawa",
    "name": "yazawa",
    "password": "NOT_LOGGING_PASSWORD",
    "shell": "/bin/bash",
    "state": "present",
    "system": false,
    "uid": 1001
}


In [29]:
!ansible -b -a 'usermod -aG wheel {admin_username}' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>



## Step 7: Starting OperationHub

ユーザ用イメージをあらかじめPullしておく。

In [37]:
!ansible -b -a 'docker pull niicloudoperation/notebook' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
Using default tag: latest
latest: Pulling from niicloudoperation/notebook
Digest: sha256:20438297076cb1997833471eba78ff50a1a5f82154d9ece0ac9a1c9a54779651
Status: Image is up to date for niicloudoperation/notebook:latest
docker.io/niicloudoperation/notebook:latest


関連イメージをビルドする。

In [30]:
!ansible -b -a 'chdir={ophub_dir} docker-compose build' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
Step 1/12 : FROM jupyterhub/jupyterhub:1.0.0
1.0.0: Pulling from jupyterhub/jupyterhub
Digest: sha256:2bd3ea9602f61aa719a4f9458c64966c9cd839145cec08f0f1980124595dbd69
Status: Downloaded newer image for jupyterhub/jupyterhub:1.0.0
 ---> 64d82994fd55
Step 2/12 : RUN apt-get update && apt-get install -y make && apt-get autoclean && apt-get clean && apt-get autoremove
 ---> Running in 9f56fd7a826f
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic InRelease [242 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [9012 B]
Get:4 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [863 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [82.2 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [977 kB]
Get:7 http://archive.ubuntu.com/ubu

サービスを起動する。

In [31]:
!ansible -b -a 'chdir={ophub_dir} docker-compose up -d' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
Creating network "jupyterhub_backend" with driver "bridge"





# 動作確認

自身のPCからVMのHTTPS (TCP 443)を許可するように設定し、以下のURLにブラウザからアクセスする。

サーバ証明書に関する警告が出るので、例外設定して確認すること。ユーザ名とパスワードを求められたら、管理者ユーザ設定で実施したユーザ名及びパスワードを与える。

> 下記のURLのアクセスがタイムアウトになる場合は、Security GroupでHTTPSが許可されていない可能性が高い...

In [32]:
print('https://' + server_name)

https://ec2-52-42-214-180.us-west-2.compute.amazonaws.com


サービスの状態・ログは以下から確認することができます。

In [33]:
!ansible -b -a 'chdir={ophub_dir} docker-compose ps' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
       Name                     Command               State               Ports            
-------------------------------------------------------------------------------------------
ophub_jupyterhub_1   jupyterhub                       Up      8000/tcp                     
ophub_proxy_1        /bin/bash -c envsubst '$SE ...   Up      80/tcp, 0.0.0.0:443->8443/tcp


In [36]:
!ansible -b -a 'chdir={ophub_dir} docker-compose logs jupyterhub' {target_group}

ec2-52-42-214-180.us-west-2.compute.amazonaws.com | CHANGED | rc=0 >>
Attaching to ophub_jupyterhub_1
jupyterhub_1  | [I 2020-07-02 02:38:04.758 JupyterHub tracking_server:44] Writing server signature to /srv/jupyterhub/data/server_signature
jupyterhub_1  | [I 2020-07-02 02:38:04.790 JupyterHub app:2120] Using Authenticator: ophubauthenticator.OphubPAMAuthenticator
jupyterhub_1  | [I 2020-07-02 02:38:04.790 JupyterHub app:2120] Using Spawner: dockerspawner.dockerspawner.DockerSpawner-0.12.0.dev
jupyterhub_1  | [I 2020-07-02 02:38:04.818 JupyterHub app:1302] Writing cookie_secret to /srv/jupyterhub/jupyterhub_cookie_secret
jupyterhub_1  | [I 2020-07-02 02:38:04.861 alembic.runtime.migration migration:130] Context impl SQLiteImpl.
jupyterhub_1  | [I 2020-07-02 02:38:04.862 alembic.runtime.migration migration:137] Will assume non-transactional DDL.
jupyterhub_1  | [I 2020-07-02 02:38:04.871 alembic.runtime.migration migration:356] Running stamp_revision  -> 4dc2d5a8c53c
jupyterhub_1  | [I